In [ ]:
import pandas as pd


In [ ]:
csv_path_happiness = "Resources/2017_happiness.csv"
csv_path_libraries = "Resources/lmw_data_09_Feb_2021.csv"

happiness_df = pd.read_csv(csv_path_happiness)
libraries_df = pd.read_csv(csv_path_libraries, encoding='latin-1')

In [ ]:
happiness_df.head()

In [ ]:
libraries_df = libraries_df.loc[(libraries_df["Data Year"] == "2017"), :]
libraries_df = libraries_df.dropna()

#libraries_df.drop(libraries_df[libraries_df["Metric Data"]=="NaN"].index, inplace=True)

#library_df = libraries_df.drop(libraries_df[libraries_df["Metric Data"] == "NaN"].index, inplace=False)

#test_df = libraries_df.loc[libraries_df["Metric Data"]=="NaN", :]
libraries_df

#test_df

In [ ]:
combined_df = pd.merge(happiness_df, libraries_df, how='left', left_on=["Country"], right_on=["Country or Area"])
combined_df

In [ ]:
combined_df = combined_df.dropna()
combined_df.head()

In [ ]:
library_visits_df = libraries_df.loc[libraries_df["Metric"]=="Visits", :]

#library_visits_df.dropna()
library_visits_df["Metric Data (INT)"] = library_visits_df["Metric Data"].astype(int)


#library_visits_df.head(100)
type_group_df = library_visits_df.groupby(["Library Type"]).sum()

type_group_df

In [ ]:
# PAM CODE HERE - start

In [ ]:
# PAM CODE HERE - end

In [ ]:
# ZOE CODE HERE - start

In [ ]:
# ZOE CODE HERE - end

In [ ]:
# MALLORY CODE HERE - start

In [ ]:
# MALLORY CODE HERE - end

In [ ]:
#ELLIE CODE HERE - start
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
libraries_df["Metric Data"] = pd.to_numeric(libraries_df["Metric Data"],errors="coerce")

In [ ]:
#create DF to show only country and number of visits?
visits_df = libraries_df.loc[libraries_df["Metric"]=="Visits", :]
visits_df.dtypes

In [ ]:
grouped_visits_df = visits_df.groupby(['Country or Area'])
print(grouped_visits_df)
grouped_visits_df=grouped_visits_df.sum().head(20)


In [ ]:
visit_number=grouped_visits_df.groupby(['Country or Area'])
new_visits_df=visit_number.sum()
new_visits_df = pd.DataFrame(grouped_visits_df["Metric Data"])
new_visits_df


In [ ]:
x_axis = np.arange(len(new_visits_df))
tick_locations = [value+0.5 for value in x_axis]

In [ ]:
# Library Visits per Country - Bar Graph 
plt.figure(figsize = (20,20))

plt.bar(x_axis,new_visits_df["Metric Data"],color='b',alpha=0.5, align='edge')
plt.xticks(tick_locations, new_visits_df.index,rotation="vertical")

plt.xlabel ("Country")
plt.ylabel ("Visits")
plt.title ("Library Visits per Country") 


#plt.xlim([])
#plt.ylim([])

plt.tight_layout()
plt.show()

In [ ]:
# ELLIE CODE HERE - end